
# DeepCAD-RT training pipeline            
<img src="https://github.com/STAR-811/DeepCAD-RT-old/blob/main/images/logo-new.png?raw=true" width = "650" height = "180" align=right />
This file will demonstrate pipeline for training microscopy data using the DeepCAD-RT algorithm.<br>
The demo shows how to construct the params and call the relevant functions for training DeepCAD-RT network. In addition, it will automatically download tif file for demo training.<br>

More information can be found in the companion paper.
See inside for details.

In [1]:
from deepcad.train_collection import training_class
from deepcad.movie_display import display
from deepcad.utils import get_first_filename,download_demo

### Select file(s) to be processed (download if not present)
The `download_demo` function will download the specific file for you and return the complete path to the file which will be stored in your `datasets` directory. If you adapt this demo for your own data make sure to pass the datasets folder name of your file(s).

In [2]:
download_demo_file = False
if download_demo_file:
    file_name='fish_localbrain' # select the demo file  to be trained (e.g. 'ATP_3D', 'fish_localbrain', 'NP_3D', ...)
    datasets_path, _ =download_demo(download_filename=file_name)
else:
    datasets_path = 'datasets/simulate_-2.51dBSNR_1000frames_demo'  # folder containing files for training

### Set up some parameters for training
Default setting shows the parameters suitable for demo file. You can change the training parameters accroding to your training data and device. For supervising the training process, you can set the flag for visualization and result storage to `True`.

In [3]:
n_epochs = 5               # number of training epochs
GPU = '0'                   # the index of GPU you will use for computation (e.g. '0', '0,1', '0,1,2')
train_datasets_size = 6000   # datasets size for training (how many patches)
patch_xy = 150              # the width and height of 3D patches
patch_t = 150               # the time dimension of 3D patches
overlap_factor = 0.4        # the overlap factor between two adjacent patches
pth_dir = './pth'           # pth file and result visualization path
num_workers = 0             # if you use Windows system, set this to 0.

# Setup some parameters for result visualization during training period (optional)
visualize_images_per_epoch = True  # choose whether to show inference performance after each epoch
save_test_images_per_epoch = True  # choose whether to save inference image after each epoch in pth path

###  Play the demo noisy movie  (optional)
Play the first noisy movie (optional). This will require loading the movie in memory which in general is not needed by the pipeline. Displaying the movie uses the OpenCV library. 

In [4]:
display_images = True

if display_images:
    display_filename = get_first_filename(datasets_path)
    print('\033[1;31mDisplaying the first raw file -----> \033[0m')
    print(display_filename)
    display_length = 300  # the frames number of the noise movie
    # normalize the image and display
    display(display_filename, display_length=display_length, norm_min_percent=1, norm_max_percent=98)

Displaying the first raw file -----> 
datasets/simulate_-2.51dBSNR_1000frames_demo/simulate_-2.51dBSNR_1000frames.tif


### Create a training class object with the specified parameters
You will creat a parameters object by passing all the parameters as a single dictionary. Parameters not defined in the dictionary will assume their default values.

In [5]:
train_dict = {
    # dataset dependent parameters
    'patch_x': patch_xy,                # you can change these params if use anisotropy patch size
    'patch_y': patch_xy,
    'patch_t': patch_t,
    'overlap_factor':overlap_factor,
    'scale_factor': 1,                   # the factor for image intensity scaling
    'select_img_num': 1000000,           # select the number of images used for training (use all frames by default)
    'train_datasets_size': train_datasets_size,
    'datasets_path': datasets_path,
    'pth_dir': pth_dir,
    
    # network related parameters
    'n_epochs': n_epochs,
    'lr': 0.00005,                       # initial learning rate
    'b1': 0.5,                           # Adam: bata1
    'b2': 0.999,                         # Adam: bata2
    'fmap': 16,                          # number of feature maps
    'GPU': GPU,
    'num_workers': num_workers,
    'visualize_images_per_epoch': visualize_images_per_epoch,
    'save_test_images_per_epoch': save_test_images_per_epoch
}

tc = training_class(train_dict)

Training parameters -----> 
{'overlap_factor': 0.4, 'datasets_path': 'datasets/simulate_-2.51dBSNR_1000frames_demo', 'n_epochs': 5, 'fmap': 16, 'output_dir': './results', 'pth_dir': './pth', 'batch_size': 1, 'patch_t': 150, 'patch_x': 150, 'patch_y': 150, 'gap_y': 90, 'gap_x': 90, 'gap_t': 90, 'lr': 5e-05, 'b1': 0.5, 'b2': 0.999, 'GPU': '0', 'ngpu': 1, 'num_workers': 0, 'scale_factor': 1, 'train_datasets_size': 200, 'select_img_num': 1000000, 'test_datasize': 400, 'visualize_images_per_epoch': True, 'save_test_images_per_epoch': True}


### Start the training process

In [6]:
tc.run()

Image list for training -----> 
Total stack number ----->  1
Noise image name ----->  simulate_-2.51dBSNR_1000frames.tif
Noise images shape ----->  (1000, 490, 490)
Using 1 GPU(s) for training -----> 
[Epoch 1/5] [Batch 208/208] [Total loss: 133879.38, L1 Loss: 431.95, L2 Loss: 267326.81] [ETA: 0:08:59] [Time cost: 134 s]      
 Testing model of epoch 1 on the first noisy file ----->
Testing image name ----->  simulate_-2.51dBSNR_1000frames.tif
Noise images shape ----->  (400, 490, 490)
 [Patch 100/100] [Time Cost: 15 s] [ETA: 0 s]      
 Displaying the first denoised file ----->

[Epoch 2/5] [Batch 208/208] [Total loss: 146795.33, L1 Loss: 447.59, L2 Loss: 293143.06] [ETA: 0:06:48] [Time cost: 307 s]      
 Testing model of epoch 2 on the first noisy file ----->
Testing image name ----->  simulate_-2.51dBSNR_1000frames.tif
Noise images shape ----->  (400, 490, 490)
 [Patch 100/100] [Time Cost: 14 s] [ETA: 0 s]      
 Displaying the first denoised file ----->

[Epoch 3/5] [Batch 208/20